# Eye Detector Algorithm

Input: An image of a face/cropped face/eye of a person

Output: The zoomed images of the eyes with a shape of 150x150 px

#### First, calling the needed modules

In [1]:
import dlib # Using pretrained models from dlib to detect the eyes
import cv2
import matplotlib.pyplot as plt
import os
import glob

## Defining Functions

### The eye cropping function for a full face input

In [2]:
def full_face_eye_cropping(img, M, N):
    """
    
    This is the function to crop and save the eye for an input of a full face
    
    Input:
    img = The input image loaded with dlib
    M, N = The landmark region of intrest ((42,46) for left eye and (36,42) for right eye)
    
    Output:
    eye = The cropped image of the input where only the eye is present in the size of (150,150)
    
    """
    
    # Defining Pre-Trained Models
    face_detector = dlib.get_frontal_face_detector()
    landmark_detector = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    
    # Detecting the faces in the input
    faces = face_detector(img, 1)
    face = faces[0]
    
    # Detecting landmarks in the image
    base_img = img.copy()
    landmarks_tuple = []
    landmarks = landmark_detector(img,face)

    for i in range(0, 68):
        x = landmarks.part(i).x
        y = landmarks.part(i).y

        landmarks_tuple.append((x, y))
            
        cv2.circle(base_img, (x, y), 2, (255, 255, 255), -1)
    
    # Creating a route between landmarks
    routes = [i for i in range(M,N)] + [M]
    route_coordinates = []
    base_img = img.copy()

    # Filling route_coordinates
    for i in range(0, len(routes) - 1):
        source_point = routes[i]
        target_point = routes[i+1]
    
        source_coordinate = landmarks_tuple[source_point]
        target_coordinate = landmarks_tuple[target_point]
    
        route_coordinates.append(source_coordinate)
    
        cv2.line(base_img, source_coordinate, target_coordinate, (255, 255, 255), 2)
        
    # Getting the eye bounding box using route_coordinates
    x_coords = [coord[0] for coord in route_coordinates]
    y_coords = [coord[1] for coord in route_coordinates]
    x_min, x_max = min(x_coords), max(x_coords)
    y_min, y_max = min(y_coords), max(y_coords)

    # Add padding
    padding = 10
    x_min = max(0, x_min - padding)
    y_min = max(0, y_min - padding)
    x_max = min(img.shape[1], x_max + padding)
    y_max = min(img.shape[0], y_max + padding)

    # Crop and resize
    eye_region = img[y_min:y_max, x_min:x_max]
    eye = cv2.resize(eye_region, (150, 150))
    
    return eye

### File saving function

In [3]:
def save_file(img, file_name, output_directory, M):
    """
    
    This function is used to save the processed image to an output directory
    
    Input:
    img = An image file loaded with dlib
    file_name = The original file name
    output_directory = The directory where the output image is set to be saved
    M = The starting point of intrest landmark (42 for left eye, 36 for right eye)
    
    Output:
    {file_name}_{left/right}_eye.png = A saved .png file of the original file's left or right eye
    
    """
    
    # Seperating for different eye cases
    if M == 42: # left eye
        new_file_name = os.path.join(output_directory,f"{os.path.splitext(file_name)[0]}_left_eye.png") 
        cv2.imwrite(new_file_name, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
    elif M == 36: # right eye
        new_file_name = os.path.join(output_directory,f"{os.path.splitext(file_name)[0]}_right_eye.png") 
        cv2.imwrite(new_file_name, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
    else: # No faces processed
        new_file_name = os.path.join(output_directory,f"{os.path.splitext(file_name)[0]}.png") 
        cv2.imwrite(new_file_name, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

## The Main Function

In [6]:
def cropped_eye(file_directory, file_name, output_directory):
    """
    
    This is the main function to detect and crop the input image
    
    Input:
    file_directory = The directory where the file is located
    file_name = the name of the file set to be processed in image format (.jpg, .png, and the like)
    output_directory = The directory where the output image is saved
    
    Output:
    {file_name}_left_eye.png = A .png file of the input image's left eye with a size of (150,150)
    {file_name}_right_eye.png = A .png file of the input image's right eye with a size of (150,150)
    
    """
    
    # Defining Pre-Trained Models
    face_detector = dlib.get_frontal_face_detector()
    landmark_detector = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")
    
    # Getting the image using file_directory and file_name
    file = file_directory + r"\\" + file_name
    
    # Loading the image and the faces
    img = dlib.load_rgb_image(file)
    faces = face_detector(img, 1)
    
    """
    Face Detection
    
    Deducing whether the input image has a detectable face, with a different processing needed if a face is not detected
    
    """
    if len(faces) >= 1: # This is when the algorithm detects a face in the input
        face = faces[0]
    
        """
        Right eye extraction
    
        Extracting the right eye image using landmarks 36-42 and saving the file
    
        """
        # Extracting the right eye
        right_eye = full_face_eye_cropping(img, 36, 42)
    
        # Saving to output_directory
        save_file(right_eye, file_name, output_directory, 36)
    
        """
        Left eye extraction
    
        Extracting the left eye image using landmarks 42-46 and saving the file
    
        """
    
        # Extracting the left eye
        left_eye = full_face_eye_cropping(img, 42, 46)
    
        # Saving to output_directory
        save_file(left_eye, file_name, output_directory, 42)
        
    else: # If no faces were detected in the input, only resize the input to 150x150 (WORK IN PROGRESS)
        eye = cv2.resize(img, (150, 150)) # Resize the input
        save_file(eye, file_name, output_directory, 0) # Save the resized file
        